<a href="https://colab.research.google.com/github/lustraka/data-analyst-portfolio-project-2022/blob/main/code/20211112_Analyze_Initial_WebPages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Import dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [30]:
# Store dataframes for further processing
from sqlalchemy import create_engine

# Upload the database from GitHub
url_db = 'https://github.com/lustraka/data-analyst-portfolio-project-2022/blob/main/data/dapp2022.db?raw=true'
r = requests.get(url_db)
with open('dapp2022.db', 'wb') as file:
  file.write(r.content)

# Create SQLAlchemy engine and connect to the database
engine = create_engine('sqlite:///dapp2022.db')

# Read dataframes from SQlite database
wp_root = pd.read_sql('SELECT * FROM wp_root', engine)
wp_root.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      120 non-null    int64 
 1   id         120 non-null    object
 2   title      120 non-null    object
 3   wp_url     120 non-null    object
 4   status     120 non-null    int64 
 5   status_ts  120 non-null    object
 6   text       120 non-null    object
 7   text_len   120 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 7.6+ KB


In [31]:
wp_root.status.value_counts()

200    103
403     10
406      2
503      1
500      1
429      1
418      1
404      1
Name: status, dtype: int64

**HTTP Status Codes**

Number | Meaning
-|-
200 | OK
403 | Forbidden
404 | Not Found
406 | Not Acceptable
418 | I'm a teapot (RFC 2324, RFC 7168)
429 | Too Many Requests (RFC 6585)
500 | Internal Server Error
503 | Service Unavailable

In [32]:
wp_root.query('status == 200 and text_len == 0')['wp_url'].to_numpy()

array(['https://www2.deloitte.com/content/dam/Deloitte/us/Documents/financial-services/us-enabling-project-success-through-analytics-placemat.pdf',
       'https://www.oracle.com/a/ocom/docs/dc/ne/aconexplaybookapril2018finalweb.pdf'],
      dtype=object)

In [33]:
wp_root.query('text_len > 0').shape[0]

101

**Conclusion** (2021-11-12) : Out of 120 web pages, 103 could be successfully loaded by the `requests.get()` function. Two URLs pointed to PDF files and couldn't be parsed with a html parser. So we ended up with 101 web pages for automated text processing. All pages are accessible via a browser, though.

**Caveat**: The results can differ when run again another time!

In [34]:
# Check selected records
wp_root.query('status == 404')

,index,id,title,wp_url,status,status_ts,text,text_len
81,81,WP82,Data Analyst - Portfolio - OVO Group Careers,https://careers.ovo.com/data-analyst-portfolio...,404,2021-11-11 15:57:25.853109,,0


In [44]:
def get_site(url):
  """Return site from URL address."""
  return re.search('://(.*?)/', url).group(1)

In [45]:
# Report results in markdown
# for _, row in wp_root.iterrows():
#   print(f'- [{row[1]}] [{row[2]}]({row[3]}) ({row[4]} | {row[7]})')

In [52]:
# Prepare a markdown table for manual data collection
# print('| id | title | site | wp_cat | wp_published | wp_updated | authors | notes |')
# print('| - | - | - | - | - | - | - | - |')
# for _, row in wp_root.iterrows():
#   print(f'| {row[1]} | {row[2][:20]}... | {get_site(row[3])} | - | - | - | - | - |')